In [38]:
from bs4 import BeautifulSoup
import requests
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np

In [39]:
def CromaScrapper(product):
    final_data=[]
    url=f"https://www.croma.com/searchB?q={product.replace(' ','%20')}%3Arelevance&text={product.replace(' ','%20')}"
    options=webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115 Safari/537.36")
    driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
    driver.get(url)
    time.sleep(5)
    link_list=[]
    links=driver.find_elements(By.XPATH,f"//div[@class='product-info']/div/div/h3/a")
    for link in links:
        link_list.append(link.get_attribute('href'))
    for link in link_list:
        
        driver.get(link)
        time.sleep(5)
        #Product name
        try:
            product_name=driver.find_element(By.CLASS_NAME,'pd-title').text.strip()
        except:
            product_name=product
    
        #Ratings
        try:
            ratings=driver.find_element(By.CLASS_NAME,'cp-rating').text.strip().split('\n')
            rating=ratings[0]
        except:
            rating=np.NaN
    
        #Number of Reviews
        try:
            reviews=ratings[1].split(' ')
            for review in reviews:
                if "(" in review:
                    number_of_reviews=review.replace('(','')
                    break
        except:
            number_of_reviews=np.NaN
    
        #Current Price
        try:
            price=driver.find_element(By.CLASS_NAME,'new-price').text.strip().split('\n')[0]
        except:
            price=np.NaN
    
        #Current Discount
        try:
            discount=driver.find_element(By.CLASS_NAME,'dicount-value').text.strip().split(' ')
            for i in discount:
                if '%' in i:
                    final_discount=i
                    break
        except:
            final_discount=np.NaN
    
        #Delivery Time
        try:
            delivery_time=driver.find_element(By.CLASS_NAME,'del-date').text.strip()
        except:
            delivery_time=np.NaN
    
        #Description
        try:
            description=driver.find_element(By.ID,'overview_inner_container').text.strip()
        except:
            description=np.NaN
        data=['Croma',product_name,rating,number_of_reviews,price,final_discount,delivery_time,description,link]
        final_data.append(data)
    
    driver.quit()
    return final_data


In [40]:
#Flipkart - add pincode feature
#Amazon fix the function for non electrnic item
#Fix all functions to get exact product name
#Fix croma scrapper

In [41]:
def MeeshoScrapper(product):
    final_data=[]
    url=f"https://www.meesho.com/search?q={product.replace(' ','%20')}"
    options=webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-infobars")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115 Safari/537.36")
    driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
    driver.get(url)
    time.sleep(10)
    
    products=driver.find_elements(By.XPATH, "//div[contains(@class,'ProductListItem__GridCol')]")
    for product in products:
        try:
            product_name=product.find_element(By.XPATH, ".//div[contains(@class, 'NewProductCardstyled__ProductHeaderWrapper')]").text
        except:
            product_name=np.NaN
        try:
            product_price=product.find_element(By.XPATH, ".//div[contains(@class, 'NewProductCardstyled__PriceRow')]").text.strip().split('\n')[0]
        except:
            product_price=np.NaN
        try:
            discount=np.NaN
            product_details=product.find_element(By.XPATH, ".//div[contains(@class, 'NewProductCardstyled__PriceRow')]").text.strip().split('\n')
            for i in product_details:
                if '%' in i:
                    discount=i
        except:
            discount=np.NaN
            
            
        try:
            product_rating=product.find_element(By.XPATH, ".//div[contains(@class, 'NewProductCardstyled__RatingsRow')]").text.strip().split()[0]
        except:
            product_rating=np.NaN
        try:
            number_of_reviews=product.find_element(By.XPATH, ".//div[contains(@class, 'NewProductCardstyled__RatingsRow')]").text.strip().split()[1]
        except:
            number_of_reviews=np.NaN
        try:
            product_link = product.find_element(By.TAG_NAME, "a").get_attribute("href")
        except:
            product_link=np.NaN

        data=['Meesho',product_name,product_rating,number_of_reviews,product_price,np.NaN,np.NaN,np.NaN,product_link]
    
        final_data.append(data)
    driver.quit()
    return final_data

In [42]:
#Bigbasket Scrapper
def BigbasketScrapper(product):
    final_data=[]
    url=f"https://www.bigbasket.com/ps/?q={product.replace(' ','+')}"
    options=webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-infobars")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115 Safari/537.36")
    driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
    driver.get(url)
    time.sleep(5)
    links=driver.find_elements(By.XPATH,"//div[contains(@class,'SKUDeck___StyledDiv')]//a")
    link_list=[]
    for link in links:
        new_link=link.get_attribute('href')
        link_list.append(new_link)
    for link in link_list:
        driver.get(link)
        time.sleep(5)
        try:
            product_name=driver.find_element(By.XPATH,"//h1[contains(@class,'Description___StyledH')]").text.strip()
        except:
            product_name=product
        try:
            product_price=driver.find_element(By.XPATH,"//td[contains(@class,'Description___StyledTd')]").text.strip()
        except:
            product_price=np.NaN
        try:
            description=driver.find_element(By.XPATH,"//div[contains(@class,'MoreDetails___StyledDiv')]").text.strip()
        except:
            decsription=np.NaN
        try:
            discount=driver.find_element(By.XPATH,"//td[@class='text-md text-appleGreen-700 font-semibold p-0']").text.strip()
        except:
            discount=np.NaN
        data=['BigBasket',product_name,np.NaN,np.NaN,product_price,discount,np.NaN,description,link]
        final_data.append(data)
    
    driver.quit()
    return final_data

In [43]:
def JiomartScrapper(product,pincode):
    final_data=[]
    url=f"https://www.jiomart.com/search/{product.replace(' ','%20')}"
    options=webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-infobars")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115 Safari/537.36")
    driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
    driver.get(url)
    cross=driver.find_element(By.ID,'btn_location_close_icon')
    cross.click()
    time.sleep(5)
    location_placeholder=driver.find_element(By.NAME,'rel_pincode')
    location_placeholder.send_keys(pincode)
    time.sleep(2)
    location_placeholder.send_keys(Keys.ENTER)
    time.sleep(4)
    link_list=[]
    links=driver.find_elements(By.XPATH,"//li[contains(@class,'ais-InfiniteHits-item')]//a")
    for link in links:
        new_link=link.get_attribute('href')
        link_list.append(new_link)
    
    for link in link_list:
        driver.get(link)
        try:
            product_name=driver.find_element(By.ID,'pdp_product_name').text.strip()
        except:
            product_name=product
        try:
            full_stars = len(driver.find_elements(By.XPATH, "//img[contains(@src, 'web-star-filled-icon.svg')]"))
            half_stars = len(driver.find_elements(By.XPATH, "//img[contains(@src, 'web-half-star-filled-icon.svg')]"))
            rating_value = full_stars + 0.5 * half_stars
        except:
            rating_value=np.NaN
        try:
            number_of_reviews=driver.find_element(By.XPATH,"//span[contains(@class,'review-count')]").text.strip()
        except:
            number_of_reviews=np.NaN
        try:
            price_details=driver.find_element(By.XPATH,"//div[contains(@class,'product-price')]").text.strip().split('\n')[0]
        except:
            price_details=np.NaN
        try:
            discount=driver.find_element(By.XPATH,"//div[contains(@class,'product-price')]").text.strip().split('\n')[1]
        except:
            discount=np.NaN
        try:
            delivery=driver.find_element(By.XPATH,"//div[contains(@class,'product-delivery-to-between')]").text.strip()
        except:
            delivery=np.NaN
        try:
            description=driver.find_element(By.XPATH,"//div[contains(@id,'pdp_description')]").text.strip()
        except:
            description=np.NaN
        data=['Jiomart',product_name,rating_value,number_of_reviews,price_details,discount,delivery,description,link]
        final_data.append(data)
    driver.quit()
    return final_data

In [44]:
def MyntraScrapper(product,pincode):
    final_data=[]
    url=f"https://www.myntra.com/{product.replace(' ','-')}?rawQuery={product.replace(' ','%20')}"
    options=webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115 Safari/537.36")
    driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
    driver.get(url)
    time.sleep(10)
    link=driver.find_elements(By.XPATH,f"//li[@class='product-base']/a")
    link_list=[]
    for links in link:
        link_list.append(links.get_attribute('href'))
    count=0
    for link in link_list:
        driver.get(link)
        if count==0:
            
            input_element=driver.find_element(By.CLASS_NAME,'pincode-code')
            input_element.send_keys(pincode)
            time.sleep(2)
            button=driver.find_element(By.CLASS_NAME,'pincode-check')
            button.click()
            time.sleep(5)
            count=count+1
        #Product name
        try:
            product_name=driver.find_element(By.CLASS_NAME,'pdp-name').text.strip()
        except:
            product_name=product
        
        #Rating
        try:
            rating=driver.find_element(By.CLASS_NAME,'index-overallRating').text.split()[0]
        except:
            rating=np.NaN
        
        #Reviews
        try:
            number_of_reviews=driver.find_element(By.CLASS_NAME,'index-ratingsCount').text.split(' ')[0]
        except:
            number_of_views=np.NaN
        
        #Price
        price=driver.find_element(By.CLASS_NAME,'pdp-price').text.strip()
        
        #Discount
        try:
            discount_raw = driver.find_element(By.CLASS_NAME, 'pdp-discount').text.strip().split()
            final_discount = next((i.replace('(', '') for i in discount_raw if "%" in i), np.NaN)
        except:
            final_discount=np.NaN
        
        #Delivery
        try:
            delivery=driver.find_element(By.CLASS_NAME,'pincode-serviceabilityTitle').text.strip()
        except:
            delivery=np.NaN
        
        #Description
        try:
            description=driver.find_element(By.CLASS_NAME,'pdp-product-description-content').text.strip()
        except:
            description=np.NaN
        
        data=['Myntra',product_name,rating,number_of_reviews,price,final_discount,delivery,description,link]
        final_data.append(data)
    driver.quit()
    return final_data

In [45]:
def FlipkartScrapper(product):
    final_data=[]
    url=f'https://www.flipkart.com/search?q={product.replace(' ','%20')}'
    options=webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
    driver.get(url)
    time.sleep(5)
    links=driver.find_elements(By.XPATH,f"//div[@class='_75nlfW']/div/div/a")
    link_list=[]
    for link in links:
        link_list.append(link.get_attribute('href'))
    for link in link_list:
        
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115 Safari/537.36"
        }
        response=requests.get(link,headers=headers)
        soup=BeautifulSoup(response.text,'html.parser')

        try:
            product_name=soup.find('h1',class_='_6EBuvT').text.strip()
        except:
            product_name=product
    
        #Ratings
        try:
            rating=soup.find('div',class_='XQDdHH').text.strip()
        except:
            rating=np.NaN
    
        #Number of Reviews
        try:
            number_of_reviews=soup.find('span',class_='Wphh3N').span.text.strip().replace('Ratings','').split(' ')[0]
        except:
            number_of_reviews=np.NaN
    
        #Current Discount
        try:
            current_discount=soup.find('div',class_='UkUFwK').span.text.strip().split()[0]
        except:
            current_discount=np.NaN
    
        #Current Price
        try:
            current_price=soup.find('div',class_='Nx9bqj').text.strip()
        except:
            current_price=np.NaN
    
        #Delivery Time
        try:
            #delivery=soup.find('div',class_='yiggsN').text.strip()
            #delivery=soup.find('span',class_='Y8v7F1').text.strip()
            delivery=soup.find('div',class_='hVvnXm').text.strip().replace('?','')
            
        except:
            delivery=np.NaN
    
        #Description
        try:
            description=soup.find('span',class_='yN+eNk').text.strip()
        except:
            description=np.NaN
    
        data=['Flipkart',product_name,rating,number_of_reviews,current_price,current_discount,delivery,description,link]
        final_data.append(data)
    driver.quit()
    return final_data

In [46]:
#Debugging errors and fixing glitches

In [47]:
def AmazonScrapper(product):
    final_data=[]
    options=webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
    url=f"https://www.amazon.in/s?k={product.replace(' ', '+')}"
    driver.get(url)
    time.sleep(5)
    link_list=[]
    links=driver.find_elements(By.XPATH,f"//span[@class='rush-component']//a")
    for link in links:
        link_list.append(link.get_attribute('href'))
    #driver.quit()
    for i in range(min(10,len(link_list))):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115 Safari/537.36"
        }
        #response = requests.get(link, headers=headers)
        link=link_list[i]
        driver.get(link)
        time.sleep(10)
        #soup=BeautifulSoup(response.text,'html.parser')
        #Product name
        try:
            product_name=driver.find_element(By.ID,'productTitle').text.strip()
        except:
            product_name=product
    
        #Ratings
        try:
            customer_reviews=driver.find_element(By.ID,'averageCustomerReviews').text.strip().split()
            rating=float(customer_reviews[0])
        except:
            rating=np.NaN
    
        # Number of reviews
        try:
            number_of_reviews=driver.find_element(By.ID,'acrCustomerReviewText').text.strip().split()
            number_of_reviews=number_of_reviews[0]
        except:
            number_of_reviews=np.NaN
    
        #Current Discount
        try:
            price_details=driver.find_element(By.ID,'corePriceDisplay_desktop_feature_div').text.strip().split()
            for details in price_details:
                if '%'in details:
                    current_discount=details
                    break
        except:
            current_discount=np.NaN
    
        #Current Price
        try:
            price_details=driver.find_element(By.ID,'corePriceDisplay_desktop_feature_div').text.strip().split()
            for details in price_details:
                if '₹'in details or '$' in details:
                    current_price=details
                    break
        except:
            current_price=np.NaN
    
        #Delivery Time
        try:
            delivery_time=driver.find_element(By.ID,'deliveryBlockMessage').text.strip()
        except:
            delivery_time=np.NaN
            
        #Description
        try:
            description=driver.find_element(By.ID,'featurebullets_feature_div').text.strip().replace('About this item','').strip()
        except:
            try:
                description=driver.find_element(By.ID,'productDescription').text.strip()
            except:
                description=np.NaN
        
            
        data=['Amazon',product_name,rating,number_of_reviews,current_price,current_discount,delivery_time,description,link]
        final_data.append(data)
    driver.quit()
    return final_data

In [48]:
#Functions
#MyntraScrapper(product,pincode)
#AmazonScrapper(product)
#FlipkartScrapper(product)
#JiomartScrapper(product,pincode)
#BigbasketScrapper(product)
#CromaScrapper(product)
#MeeshoScrapper(product)
#take_inputs()
#vendors()

In [49]:
def create_datasheet(product,vendor_list,pincode):
    final_data=[]
    for vendor in vendor_list:
        if vendor.lower().strip()=='meesho':
            data=MeeshoScrapper(product)
        elif vendor.lower().strip()=='myntra':
            data=MyntraScrapper(product,pincode)
        elif vendor.lower().strip()=='amazon':
            data=AmazonScrapper(product)
        elif vendor.lower().strip()=='flipkart':
            data=FlipkartScrapper(product)
        elif vendor.lower().strip()=='jiomart':
            data=JiomartScrapper(product,pincode)
        elif vendor.lower().strip()=='bigbasket':
            data=BigbasketScrapper(product)
        elif vendor.lower().strip()=='croma':
            data=CromaScrapper(product)
        else:
            Print(f"{vendor} is not available")
        final_data.extend(data)
    import pandas as pd
    df=pd.DataFrame(final_data,columns=['Vendor Name','Product Name','Rating','Number of Reviews','Price','Discount','Delivery','Description','Link'])
    return df

In [50]:
def take_inputs():
    product=input('Enter the product name')
    vendor_num=int(input('Enter the number of the vendors'))
    vendor_list=[]
    for i in range(vendor_num):
        vendor_list.append(input('Enter vendor name'))
    pincode=input('Enter the pincode')
    return product,vendor_list,pincode

In [51]:
product,vendor_list,pincode=take_inputs()

Enter the product name shirt for men
Enter the number of the vendors 3
Enter vendor name Amazon
Enter vendor name Flipkart
Enter vendor name Myntra
Enter the pincode 829301


In [14]:
data=AmazonScrapper('shirt for men')
data

[['Amazon',
  "DEELMO Men's Casual Button Down Shirts Long Sleeve Linen Shirt Fashion Textured Beach Summer Shirt",
  4.3,
  '293',
  '-82%',
  '₹364',
  'FREE delivery Thursday, 14 August on your first order. Order within 22 hrs 2 mins. Details',
  nan,
  'https://www.amazon.in/DEELMO-Casual-Button-Fashion-Textured/dp/B0D4M9H299/ref=sr_1_1?dib=eyJ2IjoiMSJ9.khrx1Me7s4GIQrzzJI-AkEjK502C5gf1Mbsw0O68_lICStNNc3Jwse0cks-h6U0-uQyN8yNCdtobaGNjn03aYYGzOWfbJRXBPzHh4G2A3kJPRhtb7oOAAY_6Vtyf-R4cBLFlcZaDyNnn5KNl5X4s69kigACxE1tXPvVdgoPMTvTsG7OHlJ6OotRU94brx3veNoB9Bm9QyeiUOg0-Yrol1WtNU5V6oPR1OHV5gVBQA1Iiq_ncSr2kPFzjEUlfQrF99h0xWcWUfPZCRLMHCW8fHupejXPb6R4DI_zQXIEyqR8.abB9MtxSgKXuTsFPzxuJrL4lO289BET5JCrJr8JMDb8&dib_tag=se&keywords=shirt+for+men&qid=1754899934&sr=8-1'],
 ['Amazon',
  'Highlander Men Cotton Slim Fit Shirt (PHSH000014_LT Pink/White L)',
  4.4,
  '8',
  '-72%',
  '₹499',
  'FREE delivery Tuesday, 19 August. Details',
  "Highlander Men'S Slim Fit Shirts | Checked | Long Sleeves | Spread Col

In [52]:
df=create_datasheet(product,vendor_list,pincode)
df

,Vendor Name,Product Name,Rating,Number of Reviews,Price,Discount,Delivery,Description,Link
0,Amazon,DEELMO Men's Casual Button Down Shirts Long Sl...,4.3,293,₹364,-82%,"FREE delivery Thursday, 14 August on your firs...",NaN,https://www.amazon.in/DEELMO-Casual-Button-Fas...
1,Amazon,Highlander Men Cotton Slim Fit Shirt (PHSH0000...,4.4,8,₹499,-72%,"FREE delivery Monday, 18 August. Order within ...",Highlander Men'S Slim Fit Shirts | Checked | L...,https://www.amazon.in/Highlander-Cotton-Shirt-...
2,Amazon,Men's Solid Linen Cotton Shirt | Casual | Plai...,2.9,228,₹499,-75%,"FREE delivery Tuesday, 19 August. Order within...","★NexaFlair is built on the pillars of quality,...",https://www.amazon.in/NexaFlair-Summer-Regular...
3,Amazon,U-Turn Casual Shirt for Men || Shirt for Men L...,3.5,"1,107",₹398,-75%,"FREE delivery Friday, 15 August on your first ...",,https://www.amazon.in/U-TURN-Cotton-Formal-Lig...
4,Amazon,Allen Solly Men Cotton Plain Slim Fit Shirt,3.7,"16,870",₹909,-43%,"FREE delivery Thursday, 14 August. Order withi...",Look effortlessly stylish when you don this re...,https://www.amazon.in/Allen-Solly-Geometric-Re...
...,...,...,...,...,...,...,...,...,...
95,Myntra,Men White & Green Slim Fit Printed Casual Shirt,4.3,30.5k,₹199,81%,"Get it by Tue, Aug 19","White and Green printed casual shirt, has a sp...",https://www.myntra.com/shirts/highlander/highl...
96,Myntra,Spread Collar Long Sleeves Classic Slim Fit Co...,NaN,30.5k,₹2279,40%,"Get it by Tue, Aug 19","Beige solid opaque Formal shirt ,has a spread ...",https://www.myntra.com/shirts/the+collection/t...
97,Myntra,Men White Slim Fit Solid Formal Shirt,4.3,96,₹773,57%,"Get it by Fri, Aug 15","White solid formal shirt, has a spread collar,...",https://www.myntra.com/shirts/raymond/raymond-...
98,Myntra,Checked Button-Down Collar Cotton Linen Casual...,4,3.5k,₹839,70%,"Get it by Thu, Aug 14","Navy blue, beige and red checked casual shirt,...",https://www.myntra.com/shirts/mast+%26+harbour...
